In [99]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
url = 'https://community.smartthings.com/latest'


In [100]:
import pandas as pd

In [101]:
import requests

In [102]:
import csv

filename = "Task1+.csv"
#f = open(filename, "w")
#headers = "Topic, Category, Tag, URL, Replies, Views"
#f.write(headers)

In [103]:
df = pd.DataFrame()

In [104]:
uClient = uReq(url)

In [105]:
page_html = uClient.read()

In [106]:
uClient.close()

In [107]:
page_soup = soup(page_html, "html.parser")

In [108]:
print(page_soup)

<!DOCTYPE html>

<html lang="en-US">
<head>
<meta charset="utf-8"/>
<title>Latest topics - SmartThings Community</title>
<meta content="A community for everyone interesting in making their homes and lives smarter." name="description"/>
<meta content="Discourse 2.5.0.beta5 - https://github.com/discourse/discourse version c8370b9c78df1161aa3db2699bbd53c3e04b9925" name="generator"/>
<link href="https://aws1.discourse-cdn.com/smartthings/optimized/3X/7/1/716336ba8239a791818b9d1c5b2ef237659d45a3_2_32x32.png" rel="icon" type="image/png"/>
<link href="https://aws1.discourse-cdn.com/smartthings/optimized/3X/6/0/60303ee32d0ca0a0624ac0b757ea61a67d895f34_2_180x180.png" rel="apple-touch-icon" type="image/png"/>
<meta content="#ededf5" name="theme-color"/>
<meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0, user-scalable=yes, viewport-fit=cover" name="viewport"/>
<link href="https://community.smartthings.com/latest" rel="canonical">
<script type="application/ld+json">{"@context

In [109]:
def page_search(num_page):
    base = 'https://community.smartthings.com/latest?no_definitions=true&page='
    r = requests.get(base + str(num_page)).text
    bs = soup(r, "html.parser")
    
    containers = page_soup.findAll("tr", {"class": "topic-list-item"})
    topics = []
    categories = []
    tags = []
    urls = []
    for i in containers:
        topic = i.td.a.text    
        category = i.td.div.find('span', {'class' : 'category-name'}).text      
        tag = i.td.div.div.text.replace('\n', '').strip()
        url = i.td.a.get('href')
        topics.append(topic)
        categories.append(category)
        tags.append(tag)
        urls.append(url)
        
   
    
    all_replies = page_soup.findAll("td", {'class': 'replies'})
    replies = []
    for r in all_replies:
        reply = r.text.replace('\n','').strip()
        replies.append(reply)

        
    views = []
    all_views = page_soup.findAll("td", {'class': 'views'})
    for v in all_views:
        view = v.text.replace('\n','').strip()
        views.append(view)
        
    
    dataset = pd.DataFrame({'topic': topics, 'category': categories, 'tag': tags, 'url': urls, 'reply': replies, 'views': views})
    #dataset.to_csv(filename, index = True, header = True)
    return dataset

In [110]:
df = []
for page in range(10):
    d = page_search(page)
    #d.to_csv(filename, mode='a', header=True)
    df.append(d)
df = pd.concat(df).reset_index()
df.align(df, join = 'left')
df.to_csv(filename, mode= 'w', header=True)



df

,index,topic,category,tag,url,reply,views
0,0,Public calendar for SmartThings developer events,Groups & Events,,https://community.smartthings.com/t/public-cal...,14,8817
1,1,Power Outage Causes Hub Format Error,General Discussion,,https://community.smartthings.com/t/power-outa...,5,60
2,2,Custom Capability and CLI Developer Preview,Announcements,"devicetype , ...",https://community.smartthings.com/t/custom-cap...,19,1204
3,3,'Fibaro Z-Wave FGK-101 Temperature & Door/Wind...,Community Created Device Types,"fibaro , ...",https://community.smartthings.com/t/fibaro-z-w...,527,30566
4,4,FortrezZ Water Flow Meter,Devices & Integrations,"watermeter , ...",https://community.smartthings.com/t/fortrezz-w...,379,14396
...,...,...,...,...,...,...,...
295,25,Mi Connector (Xiaomi and Yeelight),Community Created SmartApps,"smartapp_misc , ...",https://community.smartthings.com/t/mi-connect...,904,42437
296,26,"[RELEASE] Tasmota (Connect) for Sonoff, Tuya, ...",Community Created Device Types,,https://community.smartthings.com/t/release-ta...,124,5351
297,27,Rooms Manager: Smarter Rooms: Personalized hom...,Community Created SmartApps,"rooms , ...",https://community.smartthings.com/t/rooms-mana...,2117,42802
298,28,Can't get hold of support (UK),General Discussion,support,https://community.smartthings.com/t/cant-get-h...,22,147


In [111]:
#define the lists containing everything we need